## Import Dependencies

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from config2 import destination_zip, col_types

## Extract CSVs into DataFrame


In [ ]:
#Read csv into dataframe
accidents_df = pd.read_csv(destination_zip)
accidents_df.head()

In [ ]:
#Read csv into dataframe
tesla_file = 'Resources/Tesla_Deaths.csv'
tesla_df = pd.read_csv(tesla_file)
tesla_df.head(2)

## Transform U.S. Acccidents Dataframes

In [ ]:
#create a filtered dataframe from specific columns
accidents_cols = ["Date", "Country", "State", "Temperature(F)", "Day_Night"]
accidents_transformed = accidents_df[accidents_cols].copy()

#Change column names to lower case to avoid conflicts between python and sql when loading dataframes into databases
accidents_transformed = accidents_transformed.rename(columns={"Day_Night": "day_night",
                                                             "Temperature(F)": "temperature",
                                                              "Date": "date",
                                                              "State": "state",
                                                              "Country": "country"
                                                             })


accidents_transformed.dropna(how="all")

accidents_transformed.head()

## Transform Tesla Dataframes

In [ ]:
#create a filtered dataframe from specific columns
tesla_cols = ["Date", " Country ", " State ", " Deaths " ]
tesla_transformed = tesla_df[tesla_cols].copy()

#Change column names to lower case to avoid conflicts between python and sql when loading dataframes into databases
tesla_transformed = tesla_transformed.rename(columns={"Date": "date",
                                                     " Country ": "country",
                                                     " State ": "state",
                                                     " Deaths ": "deaths"
                                                     })


tesla_transformed.dropna(how="all")

tesla_transformed.head()

## Create database connection

In [ ]:
connection_string = "login:password@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
engine.table_names()

## Load DataFrames into database

In [ ]:
#Note: This took us 20-30 minutes to load into the database. It is a 25mb zip file. We did confirm it worked by running
    #query code below.
accidents_transformed.to_sql(name='us_accidents', con=engine, if_exists='append', index=True)

In [ ]:
#Note: We received an index key error when running this code that we could not debug. However, we confirmed that that data
    #did load into the database by running the query code below.
tesla_transformed.to_sql(name='tesla_deaths', con=engine, if_exists='append', index=True)

## Confirm DataFrames loaded into Database

In [ ]:
pd.read_sql_query('select * from us_accidents', con=engine).head()

In [ ]:
pd.read_sql_query('select * from tesla_deaths', con=engine).head()